In [ ]:
import json
import pandas as pd
import numpy as np

data = [json.loads(line) for line in open('../data/Ferrara2023-citizens-individualChallenges.json')]
data = [d for d in data if d['concept']['modelName'] != 'survey' and d['concept']['state'] in ['COMPLETED', 'FAILED']]
filtered_data = [
    {
        'user_id': d['playerId'],
        'counter': d['concept']['fields']['counterName'],
        'target': int(d['concept']['fields']['target']),
        'periodTarget': d['concept']['fields']['periodTarget'],
        'start': d['concept']['start'],
        'completed': d['concept']['completed'],
    }
    for d in data
]

df = pd.DataFrame(filtered_data)
df['start'] = pd.to_datetime(df['start'])

df = df[df['start'] < '2023-09-17']
df = df.groupby(['user_id', 'counter']).filter(lambda x: len(x) > 1)

df['periodTarget'] = df['periodTarget'].fillna(1).astype(int)
df['user_id'] = df['user_id'].astype('category')
df['counter'] = df.apply(lambda x: f"{x['counter']}_repetitive" if x['periodTarget'] > 1 else x['counter'], axis=1)
df['counter'] = df['counter'].astype('category')

df['discrimination'] = df['target'] * df['periodTarget']
df['discrimination'] = df.groupby('counter', observed=False)['discrimination'].transform(lambda x: x / x.max())

df.sort_values('start', ascending=True, inplace=True, ignore_index=True)

df

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

df['completed'].value_counts().plot(kind='bar', ax=ax1)
ax1.set_title('Number of challenges completed')
ax1.set_xlabel('Challenges completed')
ax1.set_ylabel('Number of challenges')

df.groupby('start').size().plot(ax=ax2)
ax3 = ax2.twiny()
df.groupby(['start', 'completed']).size().unstack().plot(ax=ax3, legend=True, stacked=False, kind='bar', color=['red', 'green'])
ax3.set_xlabel('')
ax3.set_xticks([])

ax2.set_title('Number of challenges per week')
ax2.set_xlabel('Start date')
ax2.set_ylabel('Number of challenges')

plt.show()

In [ ]:
dataset = pd.DataFrame({
    'user_id': df['user_id'].cat.codes,
    'counter': df['counter'].cat.codes,
    'discrimination': df['discrimination'],
    'completed': df['completed'].astype(int),
})

dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

torch.manual_seed(0)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

class Model(nn.Module):
    def __init__(self, n_users, n_indicator, n_hidden: int = 16, layers: list = [16, 8, 4], dropout: float = 0.2):
        super(Model, self).__init__()
        self.user_embedding = nn.Embedding(n_users, n_hidden)
        self.indicator_embedding = nn.Embedding(n_indicator, n_hidden)

        self.feedforward = nn.Sequential()
        for i, (in_layer, out_layer) in enumerate(zip([n_hidden, *layers], layers)):
            self.feedforward.add_module(f'linear_{i}', nn.Linear(in_layer, out_layer))
            self.feedforward.add_module(f'activation_{i}', nn.ReLU())
            self.feedforward.add_module(f'dropout_{i}', nn.Dropout(dropout))
        self.feedforward.add_module('output', nn.Linear(layers[-1], 1))
        
        self.fc = nn.Linear(2, 1)

    def forward(self, user, indicator, discrimination):
        user = self.user_embedding(user)
        indicator = self.indicator_embedding(indicator)
        x = user - indicator
        x = self.feedforward(x)
        x = torch.cat([x, discrimination.view(-1, 1)], dim=1)
        x = self.fc(x)
        return torch.sigmoid(x).view(-1)
    
    def fit(self, X: pd.DataFrame, y: pd.Series, epochs: int = 10, batch_size: int = 32, lr: float = 0.01):
        self.train()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion = nn.BCELoss()
        dataloader = DataLoader(Dataset(X, y), batch_size=batch_size, shuffle=False)
        
        for _ in (t := tqdm(range(epochs))):
            epoch_loss = 0
            for data in tqdm(dataloader, leave=False):
                user, indicator, discrimination, target = (d.to(DEVICE) for d in data)
                optimizer.zero_grad()
                output = self(user, indicator, discrimination)
                loss: torch.Tensor = criterion(output, target)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
            t.set_postfix({'loss': epoch_loss / len(dataloader)})
    
    @torch.no_grad()
    def predict(self, X: pd.DataFrame, batch_size: int = 32):
        self.eval()
        dataloader = DataLoader(Dataset(X), batch_size=batch_size, shuffle=False)
        predictions = torch.tensor([]).to(DEVICE)
        for data in tqdm(dataloader):
            user, indicator, discrimination = (d.to(DEVICE) for d in data)
            output = self(user, indicator, discrimination)
            predictions = torch.cat([predictions, output])
        return predictions.detach().cpu().numpy()

class Dataset(Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.Series = None):
        self.dataset = X
        self.y = y
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        X = self.dataset.iloc[idx]
        X = (
            torch.scalar_tensor(X['user_id'], dtype=torch.long),
            torch.scalar_tensor(X['counter'], dtype=torch.long),
            torch.scalar_tensor(X['discrimination'], dtype=torch.float32)
        )

        if self.y is None:
            return X
        
        return tuple([*X, torch.scalar_tensor(self.y.iloc[idx])])

n_users = dataset['user_id'].max() + 1
n_indicator = dataset['counter'].max() + 1

In [ ]:
test_df = dataset.groupby(['user_id', 'counter']).tail(1)
train_df = dataset.drop(test_df.index)

Y_hat_df = test_df[['completed']].copy()

In [ ]:
model = Model(n_users, n_indicator, n_hidden=32, dropout=0.5)
model.to(DEVICE)

model.fit(train_df[['user_id', 'counter', 'discrimination']], train_df['completed'], epochs=15, batch_size=32, lr=0.01)
Y_hat_df['model'] = model.predict(test_df[['user_id', 'counter', 'discrimination']])

Y_hat_df

In [ ]:
# Logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

X = train_df[['user_id', 'counter', 'discrimination']]
y = train_df['completed']

preprocessor = ColumnTransformer(
    transformers=[
        ('user_id', OneHotEncoder(), ['user_id']),
        ('counter', OneHotEncoder(), ['counter']),
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

pipeline.fit(X, y)

test_X = test_df[['user_id', 'counter', 'discrimination']]
test_y = test_df['completed']

Y_hat_df['logistic'] = pipeline.predict_proba(test_X)[:, 1]

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from matplotlib import pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

fpr, tpr, _ = roc_curve(Y_hat_df['completed'], Y_hat_df['model'])
ax1.plot(fpr, tpr, label=f"Model AUC: {auc(fpr, tpr):.2f}")

fpr, tpr, _ = roc_curve(Y_hat_df['completed'], Y_hat_df['logistic'])
ax1.plot(fpr, tpr, label=f"Logistic AUC: {auc(fpr, tpr):.2f}")

ax1.plot([0, 1], [0, 1], linestyle='--', color='black')
ax1.set_title('ROC curve')
ax1.set_xlabel('False positive rate')
ax1.set_ylabel('True positive rate')
ax1.legend()

precision, recall, _ = precision_recall_curve(Y_hat_df['completed'], Y_hat_df['model'])
ax2.plot(recall, precision, label=f"Model AUC: {auc(recall, precision):.2f}")

precision, recall, _ = precision_recall_curve(Y_hat_df['completed'], Y_hat_df['logistic'])
ax2.plot(recall, precision, label=f"Logistic AUC: {auc(recall, precision):.2f}")

ax2.set_title('Precision-Recall curve')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.legend()

plt.show()

In [ ]:
users = pd.Series(range(n_users), name='user_id')
counters = pd.Series(range(n_indicator), name='counter')
difficulty = pd.Series(np.linspace(0, 1, 10), name='discrimination')

query = pd.merge(users, counters, how='cross').merge(difficulty, how='cross')

query['model'] = model.predict(query)
query